<a href="https://www.kaggle.com/code/akashbiswas92/tensorflow-recommenders?scriptVersionId=93664814" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install tensorflow-recommenders


     |████████████████████████████████| 85 kB 692 kB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 14.8 MB 1.3 MB/s            
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=77049 sha256=904999faf36bbb7af3f075f891bd93122db6ff716d76122ecced76bbe447d69f
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3
  Attempting uninstall: wrapt
    Found e

In [2]:
!pip install scann

     |████████████████████████████████| 10.7 MB 543 kB/s            
     |████████████████████████████████| 497.5 MB 20 kB/s              | 430.2 MB 47.2 MB/s eta 0:00:02
     |████████████████████████████████| 462 kB 10.2 MB/s            
     |████████████████████████████████| 15.7 MB 28.2 MB/s            
     |████████████████████████████████| 5.8 MB 10.1 MB/s            
     |████████████████████████████████| 1.4 MB 26.1 MB/s            
     |████████████████████████████████| 2.1 MB 34.0 MB/s            
     |████████████████████████████████| 14.5 MB 22.4 MB/s            
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: keras
    Found existing installation: keras 2.6.0
    Uninstalli

In [3]:
import tensorflow as tf
print(tf.__version__)

import tensorflow_recommenders as tfrs
import tensorflow_datasets as tfds

import os
import pprint

from typing import Dict, Text

import pandas as pd
import numpy as np
import time


from sklearn.model_selection import train_test_split

2.8.0


In [8]:

def read_files(file_path, **kwargs):
    
    art_df_args = dict(filepath_or_buffer=file_path + 'articles.csv',low_memory = False)
    if 'art_cols' in kwargs:
        art_df_args['usecols']=kwargs['art_cols']
    
    cust_df_args = dict(filepath_or_buffer=file_path + 'customers.csv', low_memory = False)
    if  'cust_cols' in  kwargs:
        cust_df_args['usecols']=kwargs['cust_cols']
    
    trans_df_args= dict(filepath_or_buffer=file_path + 'transactions_train.csv', low_memory = False)
    if  'trans_cols' in kwargs:
        trans_df_args['usecols']=kwargs['trans_cols']
    
    art_df = pd.read_csv(**art_df_args)
    cust_df = pd.read_csv(**cust_df_args)
    trans_df= pd.read_csv(**trans_df_args)
    
    customer_lookup = cust_df.reset_index().set_index('customer_id')['index'].astype(str).to_dict()
    article_lookup =art_df.reset_index().set_index('article_id')['index'].astype(str).to_dict()
    
    trans_df['user_id']= trans_df['customer_id'].map(customer_lookup)
    trans_df['item_id']= trans_df['article_id'].map(article_lookup)
    
    unique_users = trans_df['user_id'].unique()
    unique_items = trans_df['item_id'].unique()
    
    trans_df = trans_df.drop(columns =['customer_id','article_id'])
    
    return customer_lookup, article_lookup, trans_df, unique_users, unique_items

    

In [9]:
%%time
#cust_cols=['customer_id']
#trans_cols= ['customer_id','article_id']
file_path = '../input/h-and-m-personalized-fashion-recommendations/'
customer_lookup, article_lookup, trans_data, user_vocab, item_vocab = read_files(file_path, cust_cols=['customer_id'], trans_cols= ['customer_id','article_id'])

CPU times: user 1min 4s, sys: 10.4 s, total: 1min 14s
Wall time: 1min 41s


## For the retrieval model we need -
1. Query tower - embeddings for user_ids
2. Candidate tower - embeddings for artilce_ids

Follow the steps below:

    1. Keep just the user_id and article_id
    
    2. Convert pd.DataFrame to tf.data.dataset 
    
    3. Split into train and test data
    
    4. Convert user_ids to integers and convert them embeddings visa Embedding layer
   

In [10]:
%%time
train_size =0.80
np.random.seed(1221)
train = trans_data[['user_id','item_id']].sample(frac=train_size)
test =  trans_data[['user_id','item_id']].drop(train.index)

train = tf.data.Dataset.from_tensor_slices(dict(train))
test = tf.data.Dataset.from_tensor_slices(dict(test))


2022-04-21 20:19:53.015821: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-04-21 20:19:53.030117: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


CPU times: user 31.9 s, sys: 2.79 s, total: 34.6 s
Wall time: 35 s


In [11]:
items = tf.data.Dataset.from_tensor_slices(item_vocab)

## Batches in tensorflow dataset
#### ratings.batch(1_000_000, drop_remainder = True) - this devides a tensorflow data set into equal batches of batch size = 1000000. Total unique number of user_ids are 31.78 million.

##### 31.78 million/1 million = no of batches are 32 
##### 31.78 million/1 million = no of batches are 31 if the drop_remainder is True

### tf.keras.layers.StringLookup - A preprocessing layer that maps string features to integers
### tf.keras.layers.Embedding - Turns indexes into dense vectors of fixed size



In [12]:
embedding_dimension = 32

In [13]:
## 4. Convert user_ids to integers and convert them embeddings visa Embedding layer
## Query tower

user_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
        vocabulary = user_vocab, mask_token =None),
    tf.keras.layers.Embedding(len(user_vocab)+1, embedding_dimension)])

In [14]:
## Candidate tower

item_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
        vocabulary = item_vocab, mask_token =None),
    tf.keras.layers.Embedding(len(item_vocab)+1, embedding_dimension)
])

#### This computes metrics for across top K candidates surfaced by a retrieval model.
#### The default metric is top K categorical accuracy : how often the true candidate is in in top K candidates for a given query

In [15]:
metrics = tfrs.metrics.FactorizedTopK(
    candidates=items.batch(256).map(item_model))

task = tfrs.tasks.Retrieval(metrics=metrics)

## We can put it all together in a model: (Returns user embeddings and positive item embeddings)
#### 1. User model
#### 2. Item model
#### 3. Retrieval task layer


In [16]:
class UserItemModel(tfrs.Model):
    
    def __init__(self, user_model, item_model):
        super().__init__()
        self.user_model : tf.keras.Model = user_model
        self.item_model : tf.keras.Model = item_model
        self.task : tf.keras.layers.Layer = task
            
    def compute_loss(self,features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        
        user_embeddings = self.user_model(features['user_id'])
        positive_item_embeddings = self.item_model(features['item_id'])
        
        return self.task(user_embeddings,positive_item_embeddings)

In [17]:
model = UserItemModel(user_model, item_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [19]:
cached_train = train.batch(16384).cache()
cached_test = test.batch(4096).cache()

In [ ]:
%%time
model.fit(cached_train, epochs=3)

2022-04-21 20:20:51.408920: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 610335816 exceeds 10% of free system memory.
2022-04-21 20:20:51.768425: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 610335816 exceeds 10% of free system memory.
2022-04-21 20:20:52.320552: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 610335816 exceeds 10% of free system memory.
2022-04-21 20:20:52.788445: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 610335816 exceeds 10% of free system memory.


Epoch 1/3


2022-04-21 20:20:56.294717: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1073741824 exceeds 10% of free system memory.


   3/1553 [..............................] - ETA: 52:35:02 - factorized_top_k/top_1_categorical_accuracy: 2.0345e-05 - factorized_top_k/top_5_categorical_accuracy: 1.8311e-04 - factorized_top_k/top_10_categorical_accuracy: 3.2552e-04 - factorized_top_k/top_50_categorical_accuracy: 9.7656e-04 - factorized_top_k/top_100_categorical_accuracy: 0.0017 - loss: 158991.8490 - regularization_loss: 0.0000e+00 - total_loss: 158991.8490